# Using symmetry in the problems to make quantum computing more efficient
We can use quantum computers to help us with machine learning. By utilizing variational quantum circuits, we can use quantum computers to learn patterns in data.

In order for this to work well, we need to come up with a way to encode the patterns we want to learn into the quantum computer. Especially on near-term quantum hardware, this requires quite some resources. One way to reduce these resource requirements and learn more efficiently is to take advantage of symmetries in the data we are trying to learn.

One approach is taken in [this paper](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.4.010328) (CC-BY 4.0, Johannes Jakob Meyer, Marian Mularski, Elies Gil-Fuster, Antonio Anna Mele, Francesco Arzani, Alissa Wilms, and Jens Eisert) and we will use it as a guideline for this challenge. A huge thank you to the authors for providing this idea and explaing it so well :-)

# Circuit without symmetries

In this notebook we implement the same Tic-Tac-Toe model but without taking into account the symmetries of the problem. This means that we have a different parameter for each single and two-qubit gate. Our aim is to compare the performance of both approaches.

## Building the circuit

Now it is time to build the circuit. For this, we will start with some basic imports. For todays challenge, we use qiskit. However, this can be also acchieved with PennyLane or Cirq.


### Encoding the data

The basic element needed for your first program is the QuantumCircuit. We begin by creating a `QuantumCircuit` comprised of nine qubits.

The equivariant embedding is constructed by encoding the different numerical values that represent a game via a Pauli-X rotation on separate qubits that we view in a planar grid. To distribute the three data features equidistantly, we use a multiple of 2π/3 for the rotation angle.

In [3]:
import numpy as np
from qiskit import *
from utils import *

boards, winners = load_boards(include_draws = True)

print("boards: ", boards[1:5])
print("winners: ",  winners[1:5])
print(len(boards))
print(len(winners))

# Create a Quantum Circuit acting on a quantum register of nine qubits
def make_circuit(x, params, layers):
    circ = QuantumCircuit(9)
    for i in range(layers):
        l = i*34
        encode_data(x, circ)
        # use functions for no symmetry
        ns_add_single_qubit_gates(params[l:l+18], circ)
        ns_add_two_qubit_gates(params[l+18:l+34], circ)

    return circ

layers = 1

boards:  [['x', 'x', 'x', 'x', 'o', 'o', '', 'o', ''], ['o', 'x', '', 'x', '', '', 'o', 'o', 'x'], ['o', '', 'o', 'x', 'x', 'o', 'x', '', ''], ['o', 'x', 'x', '', 'o', 'x', '', '', 'o']]
winners:  [[0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 0, 0]]
1413
1413


# Train
Now that we have our circuit, however, the results obtained are not quite what we wanted. So, we still need to train it on the given data. 

Therefore, we need to define, how far we are away from the expected result. One way to do this, is to use the l2_loss function.

In [4]:
# Define the loss function
def l2_loss(output, target):
    output, target = np.array(output), np.array(target)
    return np.sum(np.abs(output - target)**2)

Split data into test and train data

In [5]:
x = boards
y = winners

# shuffle the indices
shuffle_indices = np.random.permutation(len(x))
train_size = int(len(x) * 0.3)

# split the indices into training and testing sets
train_indices = np.array(shuffle_indices[:train_size])
test_indices = np.array(shuffle_indices[train_size:])

# create the training and testing sets
x_train, y_train = np.take(x, train_indices, axis=0), np.take(y, train_indices, axis=0)
x_test, y_test = np.take(x, test_indices, axis=0), np.take(y, test_indices, axis=0)


print("Example train data: ", x_train[17], y_train[17])

Example train data:  ['x' 'x' '' 'o' 'x' 'o' '' 'o' 'x'] [0 0 1]


In [6]:
from tqdm import tqdm
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

# Define the optimizer
from qiskit.algorithms.optimizers import SPSA
optimizer = SPSA(maxiter = 100)



# Define the cost function to be minimized by the optimizer
def cost_function(params):

    cost = 0
    # TODO: Calculate the cost and return them
    estimator = Estimator()
    observables = (
            SparsePauliOp("ZIZIZIZII") / 4.0,
            SparsePauliOp("IZIZIZIZI") / 4.0,
            SparsePauliOp("IIIIIIIIZ")
        )
    
    for x, y in zip(x_train, y_train):

        circ = make_circuit(x, params, layers)
        circuits = (
            circ,
            circ,
            circ
        )

        job = estimator.run(circuits, observables)
        result = job.result().values.tolist()
        
        cost += l2_loss(result, y)
        
    print(cost)
    return cost

# Initialize the parameters
params = np.random.rand(34*layers)*2*np.pi

# Train the circuit
print('Initial parameters:', params)

# Check the qiskit docs to figure out how to start an optimizer
result = optimizer.minimize(cost_function, x0 = params)
print("Finished optimizing.")

Initial parameters: [5.86327071 0.11985395 3.10866994 1.35522317 5.37179068 0.168139
 5.47940569 2.65101153 0.8496218  5.48818343 2.3003046  1.87901355
 5.55088962 2.85507062 2.110803   1.53965332 3.91217179 2.40975143
 4.98609325 0.53301517 4.57266451 2.58594659 0.24111943 3.99379298
 0.80089924 0.46371111 4.49781824 4.62207459 1.96949282 5.91430477
 6.15184067 3.62651275 3.0990945  0.87506631]
597.0258021063675
479.930642804722
495.2985662933062
578.3333103612254
538.6319102922549
529.1390482406719
494.36419929575425
577.3252444659319
575.9158141089938
491.2860497537219
479.57576803206734
596.7424598745738
521.5693863100095
543.2627212037536
577.1843450220243
491.81357516756884
536.8394275893872
529.7481348606384
543.5889229336243
521.3966477676358
598.4069843655661
487.44774773887394
578.6442472922968
495.69152178366767
544.0447069420213
522.8681398151637
576.9803040207471
492.0481917760122
545.1408555374106
520.2258439452953
537.0423224599848
529.5104689307484
536.9141640434348
529

## Use the model
The final model can then be used to predict the winner for any given tic-tac-toe board.

In [7]:
def predict(data, params):
    
    circ = make_circuit(data, params, layers)

    #TODO:  create observables and measure
    exp_val_o = 0
    exp_val_draw = 0
    exp_val_x = 0
    
    estimator = Estimator()
    observables = (
            SparsePauliOp("ZIZIZIZII") / 4.0,
            SparsePauliOp("IZIZIZIZI") / 4.0,
            SparsePauliOp("IIIIIIIIZ")
        )
    
    circuits = (
            circ,
            circ,
            circ
        )

    job = estimator.run(circuits, observables)
    result = job.result().values.tolist()
    exp_val_o = result[0]
    exp_val_draw = result[1]
    exp_val_x = result[2]
    
    

    return [exp_val_o, exp_val_draw, exp_val_x]

In [10]:
def test_model(params, x_data, y_data):
    total_loss = 0
    correct_guesses = 0
    i = 0
    for x, y in zip(x_data, y_data):
        pred = predict(x, params)
        pred_discrete = [0, 0, 0]
        pred_discrete[np.argmax(np.abs(pred))] = 1
        total_loss += l2_loss(pred, y)
        correct_guesses += np.all(pred_discrete == y)
        
        if i % 100 == 0 and False: 
            print("Correct output:", y)
            print("Actual output:", pred)
            print("Discrete_output:", pred_discrete)
            print("---------------")
        i+= 1

    total_loss = total_loss / len(x_data)

    print("Correct guesses: {}/{} ({}%)".format(correct_guesses, len(x_data), round(correct_guesses/len(x_data)*100)))
    print("L2 Loss: {}".format(total_loss))

print("Train data:")
test_model(result.x, x_train, y_train)

print("Test data:")
test_model(result.x, x_test, y_test)

Train data:
Correct guesses: 95/423 (22%)
L2 Loss: 0.9960939754231912
Test data:
Correct guesses: 238/990 (24%)
L2 Loss: 0.9950416678354778
